## Import Libraries


In [19]:
import os
import pytesseract
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # Example model
from sklearn.metrics import mean_squared_error

# === Step 1: Image Downloading === #

In [4]:
image_dir = 'images'
os.makedirs(image_dir, exist_ok=True)

## Load Datasets

In [10]:
# Load datasets
train_df = pd.read_csv(r"C:\Users\Meher\Downloads\Amazon-ML-Challenge-2024-main\Amazon-ML-Challenge-2024-main\dataset\train.csv")

## Download Images

In [11]:
def download_images(image_url, image_id, save_dir):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image_path = os.path.join(save_dir, f'{image_id}.jpg')
            image.save(image_path)
            print(f"Image {image_id} downloaded successfully!")
        else:
            print(f"Failed to download image {image_id}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading image {image_id}: {e}")
        # Download all images
for index, row in train_df.iterrows():
    image_url = row['image_link']
    image_id = index
    download_images(image_url, image_id, image_dir)

Image 0 downloaded successfully!
Image 1 downloaded successfully!
Image 2 downloaded successfully!
Image 3 downloaded successfully!
Image 4 downloaded successfully!
Image 5 downloaded successfully!
Image 6 downloaded successfully!
Image 7 downloaded successfully!
Image 8 downloaded successfully!
Image 9 downloaded successfully!
Image 10 downloaded successfully!
Image 11 downloaded successfully!
Image 12 downloaded successfully!
Image 13 downloaded successfully!
Image 14 downloaded successfully!
Image 15 downloaded successfully!
Image 16 downloaded successfully!
Image 17 downloaded successfully!
Image 18 downloaded successfully!
Image 19 downloaded successfully!
Image 20 downloaded successfully!
Image 21 downloaded successfully!
Image 22 downloaded successfully!
Image 23 downloaded successfully!
Image 24 downloaded successfully!
Image 25 downloaded successfully!
Image 26 downloaded successfully!
Image 27 downloaded successfully!
Image 28 downloaded successfully!
Image 29 downloaded succ

KeyboardInterrupt: 

# === Step 2: OCR and Preprocessing === #

In [22]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img_np = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    _, img_thresh = cv2.threshold(img_np, 150, 255, cv2.THRESH_BINARY)
    return Image.fromarray(img_thresh)

def extract_text_from_image(image_path):
    try:
        img = preprocess_image(image_path)
        print("done")
        return pytesseract.image_to_string(img)
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return ""

# Perform OCR on all images
ocr_results = {}
for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg"):
        image_path = os.path.join(image_dir, image_file)
        text = extract_text_from_image(image_path)
        ocr_results[image_file] = text

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done



# === Step 3: Preprocess Entity Values === #

In [26]:
import re

def preprocess_entity_value(value):
    try:
        # Ensure proper spacing between numbers and units
        value = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', value)
        
        # Check for range in list format [x, y] unit
        match = re.match(r"\[(\d+\.?\d*),\s*(\d+\.?\d*)\]\s+(\w+)", value)
        if match:
            start_number, end_number, unit = match.groups()
            return f"{start_number}-{end_number}", unit
        
        # Check for range in "x to y unit" format
        if ' to ' in value:
            start, end = value.split(' to ')
            start_number, start_unit = start.split(maxsplit=1)
            end_number, end_unit = end.split(maxsplit=1)
            
            if start_unit != end_unit:
                raise ValueError(f"Different units in range: {start_unit} and {end_unit}")
            
            return f"{start_number}-{end_number}", start_unit
        
        # Handle single value
        parts = value.split(maxsplit=1)
        if len(parts) != 2:
            raise ValueError(f"Invalid format in {value}")
        
        number = float(parts[0])
        unit = parts[1]
        return number, unit
    
    except ValueError as e:
        print(f"Error processing value '{value}': {e}")
        return None, None

# Apply the modified function
train_df[['value', 'unit']] = train_df['entity_value'].apply(preprocess_entity_value).apply(pd.Series)


# === Step 4: Encode Categorical Features and Prepare Dataset === #

In [32]:
X = pd.get_dummies(train_df[['group_id', 'entity_name']])
y = train_df['value']

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
def preprocess_target(value):
    try:
        # If the value is already a float, return it directly
        if isinstance(value, float):
            return value
        
        # Handle range values like "110.0-120.0"
        if '-' in value:
            start, end = value.split('-')
            midpoint = (float(start) + float(end)) / 2
            return midpoint
        
        # Convert single numeric string values to float
        return float(value)
    
    except ValueError as e:
        print(f"Error processing target value '{value}': {e}")
        return None

# Apply preprocessing to y_train
y_train = y_train.apply(preprocess_target)

# Verify that no None values remain in y_train
assert y_train.notnull().all(), "Some target values could not be processed."

# Preprocess y_test
y_test = y_test.apply(preprocess_target)

# Verify all values are numeric
assert y_test.notnull().all(), "Some target values in y_test could not be processed."



# === Step 5: Train the Model === #

In [37]:
# Example: Random Forest Regressor
import pickle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as 'model.pkl'")

Mean Squared Error: 7.706568392315835e+26
Model saved as 'model.pkl'



# === Step 6: Make Predictions on New Data === #

In [35]:
test_df = pd.read_csv(r"C:\Users\Meher\Downloads\Amazon-ML-Challenge-2024-main\Amazon-ML-Challenge-2024-main\dataset\test.csv")
X_test_new = pd.get_dummies(test_df[['group_id', 'entity_name']])
y_pred_new = model.predict(X_test_new)

# Add predictions to the test DataFrame
test_df['predicted_value'] = y_pred_new

# Save predictions to CSV (for competition submission)
test_df.to_csv('test_predictions.csv', index=False)